In [1]:
import pandas as pd

In [2]:
# Read targets
car=pd.read_excel('data/company_data/Data.xlsx',sheet_name='Targets_car')
OG=pd.read_excel('data/company_data/Data.xlsx',sheet_name='Targets_OG')
Airlines=pd.read_excel('data/company_data/Data.xlsx',sheet_name='Targets_Airlines')

#Read CDP data on emissions
scope3_emission=pd.read_csv('data/CDP/scope3_2023.csv')
scope1_emissions=pd.read_csv('data/CDP/scope1_2023.csv')
scope2_emissions=pd.read_csv('data/CDP/scope2_2023.csv')

#only keep most recent records
scope1_emissions=scope1_emissions[scope1_emissions['RowName']=='Reporting year']
scope2_emissions=scope2_emissions[scope2_emissions['RowName']=='Reporting year']

#make a list of relevant companies
relevant_car=list(car['CDP_ID'].unique())
relevant_OG=list(OG['CDP_ID'].unique())
relevant_Airlines=list(Airlines['CDP_ID'].unique())

#filter for relevant companies
scope3_car=scope3_emission[scope3_emission['Account number'].isin(relevant_car)]
scope3_OG=scope3_emission[scope3_emission['Account number'].isin(relevant_OG)]
scope3_Airlines=scope3_emission[scope3_emission['Account number'].isin(relevant_Airlines)]

scope1_car=scope1_emissions[scope1_emissions['Account number'].isin(relevant_car)]
scope1_OG=scope1_emissions[scope1_emissions['Account number'].isin(relevant_OG)]
scope1_Airlines=scope1_emissions[scope1_emissions['Account number'].isin(relevant_Airlines)]

scope2_car=scope2_emissions[scope2_emissions['Account number'].isin(relevant_car)]
scope2_OG=scope2_emissions[scope2_emissions['Account number'].isin(relevant_OG)]
scope2_Airlines=scope2_emissions[scope2_emissions['Account number'].isin(relevant_Airlines)]

scope3_car=scope3_car[~scope3_car['RowName'].str.contains('Other')]
scope3_og=scope3_OG[~scope3_OG['RowName'].str.contains('Other')]
scope3_Airlines=scope3_Airlines[~scope3_Airlines['RowName'].str.contains('Other')]

## Calculate average scope 3 shares

In [3]:
# make dataframe
df_scope3_by_cat=pd.DataFrame(index=scope3_car['RowName'].unique())

In [4]:
import pandas as pd

def calculate_average_share(scope3_emission):
    # Convert 'scope3_emissions' column to numeric if it's not already numeric
    scope3_emission['scope3_emissions'] = pd.to_numeric(scope3_emission['scope3_emissions'], errors='coerce')
    filtered_df = scope3_emission[~scope3_emission['evaluation_status'].isin(['Not evaluated', 'Relevant, not yet calculated'])]

    # Group data by company
    grouped = filtered_df.groupby('Organization')

    # Initialize an empty DataFrame to store results
    results = pd.DataFrame()

    # Iterate over each group (company)
    for company, group_data in grouped:
        # Calculate total Scope 3 emissions for the company
        total_scope3_emissions = group_data['scope3_emissions'].sum()

        # Calculate the share of each category in total Scope 3 emissions
        group_data['category_share'] = group_data['scope3_emissions'] / total_scope3_emissions

        # Append the results to the overall results DataFrame
        results = pd.concat([results, group_data])

    # Calculate the average share of each category across all companies
    average_share_by_category = results.groupby('RowName')['category_share'].mean()
    
    # Normalize shares to sum up to 1
    total_shares = average_share_by_category.sum()
    average_share_by_category = average_share_by_category / total_shares


    return average_share_by_category

#Calculate average share of scope 3 by category per sectors
df_scope3_by_cat['car']=calculate_average_share(scope3_car)
df_scope3_by_cat['OG']=calculate_average_share(scope3_OG)
df_scope3_by_cat['Airlines']=calculate_average_share(scope3_Airlines)


df_scope3_by_cat.to_pickle('data/temp/scope3_by_cat.pkl')



/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38801/1088518061.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scope3_emission['scope3_emissions'] = pd.to_numeric(scope3_emission['scope3_emissions'], errors='coerce')


In [5]:
df_scope3_by_cat

,car,OG,Airlines
Purchased goods and services,0.168074,0.052965,0.104937
Capital goods,0.015673,0.002256,0.039100
Fuel-and-energy-related activities (not included in Scope 1 or 2),0.003689,0.073438,0.480481
Upstream transportation and distribution,0.010740,0.029479,0.060037
Waste generated in operations,0.001383,0.002192,0.001315
Business travel,0.000456,0.000479,0.003528
Employee commuting,0.001553,0.000397,0.010461
Upstream leased assets,0.000265,0.003508,0.006380
Downstream transportation and distribution,0.005415,0.040737,0.005480
Processing of sold products,0.030173,0.049471,NaN


## Calculate share of scopes of total emissions

In [6]:
# Aggrecate scope 3 emissions by company
def get_tot_scope3(df):
    df=df.reset_index()

    df_return=pd.DataFrame(columns=['scope3_emissions'], index=df['Account number'].unique())

    
    df['scope3_emissions']=pd.to_numeric(df['scope3_emissions'], errors='coerce') 

    df_return['scope3_emissions']=df.groupby('Account number')['scope3_emissions'].sum()

    return df_return


# get share of total emissions by scope
def get_shares(scope1, scope2, scope3):
    scope1['scope1_emissions']=pd.to_numeric(scope1['scope1_emissions'], errors='coerce')
    scope2['scope2_location_based']=pd.to_numeric(scope2['scope2_location_based'], errors='coerce')

    exclude_list=scope1[scope1['scope1_emissions'].isna()].index
    exclude_list=exclude_list.append(scope2[scope2['scope2_location_based'].isna()].index)
    exclude_list=exclude_list.append(scope3[scope3['scope3_emissions']==0].index)

    scope1=scope1[~scope1.index.isin(exclude_list)]
    scope2=scope2[~scope2.index.isin(exclude_list)]
    scope3=scope3[~scope3.index.isin(exclude_list)]


    total_emissions = scope1['scope1_emissions'].sum()+scope2['scope2_location_based'].sum()+scope3['scope3_emissions'].sum()
    share_scope1=scope1['scope1_emissions'].sum()/total_emissions
    share_scope2=scope2['scope2_location_based'].sum()/total_emissions
    share_scope3=scope3['scope3_emissions'].sum()/total_emissions

    return [share_scope1, share_scope2, share_scope3]








In [7]:
#calculate share of scopes
share_car=get_shares(scope1_car, scope2_car, get_tot_scope3(scope3_car))
share_OG=get_shares(scope1_OG, scope2_OG, get_tot_scope3(scope3_OG))
share_Airline = get_shares(scope1_Airlines, scope2_Airlines, get_tot_scope3(scope3_Airlines))

df_scope_share=pd.DataFrame([share_car, share_OG, share_Airline], columns=['Scope 1', 'Scope 2', 'Scope 3'], index=['Automobile', 'O&G', 'Airline'])

df_scope_share.to_pickle('data/temp/scope_share.pkl')
scope3_emission.to_pickle('data/temp/scope3.pkl')

/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38801/2050423534.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scope1['scope1_emissions']=pd.to_numeric(scope1['scope1_emissions'], errors='coerce')
/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38801/2050423534.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scope2['scope2_location_based']=pd.to_numeric(scope2['scope2_location_based'], errors='coerce')
/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_38801/2050423

In [8]:
df_scope_share

,Scope 1,Scope 2,Scope 3
Automobile,0.005119,0.008950,0.985932
O&G,0.104084,0.009606,0.886309
Airline,0.732103,0.003224,0.264673


In [9]:
df_scope_share

,Scope 1,Scope 2,Scope 3
Automobile,0.005119,0.008950,0.985932
O&G,0.104084,0.009606,0.886309
Airline,0.732103,0.003224,0.264673


In [10]:
(df_scope3_by_cat*100)

,car,OG,Airlines
Purchased goods and services,16.807404,5.296513,10.493733
Capital goods,1.567311,0.225609,3.910003
Fuel-and-energy-related activities (not included in Scope 1 or 2),0.368939,7.343755,48.048137
Upstream transportation and distribution,1.074021,2.947943,6.003737
Waste generated in operations,0.138286,0.219184,0.131547
Business travel,0.045600,0.047949,0.352800
Employee commuting,0.155327,0.039740,1.046091
Upstream leased assets,0.026525,0.350825,0.638005
Downstream transportation and distribution,0.541510,4.073689,0.547951
Processing of sold products,3.017343,4.947130,NaN
